# PH2102 Electromagnetism

## Numerical methods and computation with Python
### Lesson 2

In [ ]:
# package(s) installation

import sys
!{sys.executable} -m pip install sympy

In [ ]:
# preamble

%matplotlib inline
from scipy import *
import sympy as sym
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

## 2. Gentle introduction to Symbolic Python<a name="sympy"></a>

For this section, you need `sympy` library installed for `python`. `sympy` deals with the computation of mathematical objects symbolically. This means that the mathematical objects are represented exactly and mathematical expressions with unevaluated variables are left in symbolic form. Before you dive headfirst into using `sympy` for all of your analytical solutions however, it is unfortunate that many of the sophisticated mathematical expressions (e.g. complex multi-dimensional differential equations) in physics remain a work in progress in `sympy` implementation. Always check `sympy`'s [online documentation](https://docs.sympy.org/latest/index.html) for an update on its current functionalities.

### A. Basic syntax

`sympy`'s functions, variables, and even floats aren't the same as `numpy`/ `scipy` analogues. For example 
>sympy.exp() $\neq$ scipy.exp()

In [ ]:
sym.exp(3)

Symbols that are going to used as symbolic variable must be declared as such.

In [ ]:
x = sym.symbols('x')
sym.exp(x)

Let take a look at assigning values to symbolic variables using the `subs` function.

In [ ]:
sym.exp(x).subs({x:3})

To evaluate the numerical value of a `sympy` expression, use the `evalf` function (alternatively, just use decimal point when declaring/assigning numerical values.)

In [ ]:
sym.exp(3).evalf(), sym.exp(3.), sym.exp(x).subs({x:3.})

`sympy` is able to convert some standard variables to LaTeX output.

In [ ]:
sym.init_printing() # for LaTeX formatted output

Sigma, Sigma_p = sym.symbols('Sigma, \Sigma^{\prime}')

Sigma, Sigma_p, sym.pi, sym.E

One consequence of `sympy`'s symbolic expressions is they must be turned into `scipy`/ `numpy` expressions if they are to be evaluated for plotting or numerical results. This is done with the `lambdify` function call.

In [ ]:
x, y = sym.symbols('x, y')
f = 12*x**3
g = 12*x**3 * sym.exp(y)

f = sym.lambdify(x, f) # turns sympy expression f into a function f(x) for use in scipy/numpy
g = sym.lambdify((x, y), g) # turns sympy expression g into function g(x,y) for use in scipy/numpy

xx = arange(-4, 4, 0.05) # xx so that it doesn't collide with symbolic x
A = f(xx)
B = g(xx, log(1/2))
plt.plot(xx, A)
plt.plot(xx, B)

### B. Vector calculus

#### Vector and scalar fields<a name="field"></a>
To define a vector field, declare a reference frame, R, using `ReferenceFrame` from `sympy.physics.vector` submodule and initialise the basis vectors with R.x, R.y and R.z respectively.

In [ ]:
from sympy.physics.vector import ReferenceFrame

R = ReferenceFrame('R')
vector_field = 3*R.x + 4*R.y + 5*R.z
vector_field

To define a scalar field, the base scalars (coordinate variables) can be initialised with R[0], R[1] and R[2] instead.

In [ ]:
R = ReferenceFrame('R')
scalar_field = 2*R[0]**2*R[1]
scalar_field

#### Gradient, divergence and curl<a name="gdc"></a>
`sympy` has helpful functions for calculating the gradient, divergence and curl of a field, using `grad`, `div` and `curl` function calls respectively.

Recall our previous example for __gradient__.

In [ ]:
from sympy.physics.vector import gradient

R = ReferenceFrame('R')
scalar_field = sym.exp(-R[0]**2 - R[1]**2)
g = gradient(scalar_field, R)
g

In [ ]:
## set up the co-ordinates
x = linspace(-2., 2., 20)
y = linspace(-2., 2., 20)
X, Y = meshgrid(x, y)

## seperating into components for the gradient
x_component = g.to_matrix(R)[0]
y_component = g.to_matrix(R)[1]

## converting to scipy/ numpy expressions
grad_x = sym.lambdify((R[0], R[1]), x_component)
grad_y = sym.lambdify((R[0], R[1]), y_component)
Z = sym.lambdify((R[0], R[1]), scalar_field)

## setting up plot
plt.gca().set_aspect('equal', adjustable='box')

## plotting the gradient (which is a vector field)
plt.contour(X, Y, Z(X, Y)) # analogous to a topographic map
plt.quiver(X, Y, grad_x(X, Y), grad_y(X, Y))

Exercise 6) Reproduce the same surface plot for our __divergence__ example above, this time using `sympy` library.

In [ ]:
from sympy.physics.vector import divergence

## complete the code here

In [ ]:
## set up the co-ordinates
x = linspace(-2., 2., 20)
y = linspace(-2., 2., 20)
X, Y = meshgrid(x, y)

## converting to scipy/ numpy expressions

## setting up plot
ax = plt.figure().gca(projection = '3d')

## plotting the divergence (which is a scalar field)


Exercise 7) Let's replicate the result from our __curl__ example using `sympy`.

In [ ]:
from sympy.physics.vector import curl

## complete the code here

In [ ]:
## set up the co-ordinates
x = linspace(-2., 2., 10)
y = linspace(-2., 2., 10)
z = linspace(0., .1, 10)
X, Y, Z = meshgrid(x, y, z, indexing='ij')

## seperating into components for the curl

## seperating into components for the original vector field

## converting to scipy/ numpy expressions

## setting up plot
ax = plt.figure().gca(projection = '3d')

## plotting the curl (which is a vector field)


#### Laplacian of 1/r<a name="lap_dirac"></a>

We see in the lectures that the laplacian of $\frac{1}{r}$ deserves special treatment, and its analytical solution is given as
<h3 align="center">$\nabla^2 \frac{1}{r}=-4\pi \delta^3(\vec{r})$</h3>

Let us see how does `sympy` handles such a situation.

In [ ]:
R = ReferenceFrame('R')
scalar_field = 1/sym.sqrt(R[0]**2+R[1]**2+R[2]**2)
g = gradient(scalar_field, R)
f = divergence(g, R)
sym.simplify(f)

_*Attention: We need to note that `sympy` correctly gives us $0$ for $r \neq 0$, but does not recognise the solution to $\nabla^2 \frac{1}{r}$ at the singularity point $r=0$._

#### Conservative vector field<a name="conserv"></a>

A vector field is conservative when it is the gradient of some scalar field. The line integral of a conservative vector field is independent of the path but depends only on the end-points. A conservative vector field is also said to be ‘irrotational’, since the curl of a conservative field is always zero. To check if a vector field is conservative, we can just call the `is_conservative` function from `sympy.physics.vector` submodule without explicitly solving for its curl.

In [ ]:
from sympy.physics.vector import is_conservative

R = ReferenceFrame('R')
vector_field = R[1]*R[2]*R.x + R[0]*R[2]*R.y + R[0]*R[1]*R.z
is_conservative(vector_field)

#### Solenoidal vector field<a name="solen"></a>
A vector field is solenoidal when its divergence is zero at all points in space. To check if a vector field is solenoidal, we call the `is_solenoidal` function from `sympy.physics.vector` submodule.

In [ ]:
from sympy.physics.vector import is_solenoidal

R = ReferenceFrame('R')
vector_field = R[1]*R[2]*R.x + R[0]*R[2]*R.y + R[0]*R[1]*R.z
is_solenoidal(vector_field)

####  Scalar potential function<a name="scapot"></a>
A conservative vector field is the gradient of some scalar field. This scalar field is called the scalar potential field. The `scalar_potential` function in `sympy.physics.vector` submodule calculates the scalar potential field corresponding to a given conservative vector field (sans the extra constant of integration).

In [ ]:
from sympy.physics.vector import scalar_potential

R = ReferenceFrame('R')
cons_field = 4*R[0]*R[1]*R[2]*R.x + 2*R[0]**2*R[2]*R.y + 2*R[0]**2*R[1]*R.z
scalar_potential(cons_field, R)

#### Scalar potential difference<a name="scapotdiff"></a>
The line integral with respect to a conservative vector field depends not on the path but the end-points of the path. This can be easily computed with `scalar_potential_difference`.

In [ ]:
from sympy.physics.vector import Point
from sympy.physics.vector import scalar_potential_difference

R = ReferenceFrame('R')
O = Point('O') # origin
P = O.locatenew('P', 1*R.x + 2*R.y + 3*R.z) # new point from origin
vectfield = 4*R[0]*R[1]*R.x + 2*R[0]**2*R.y
scalar_potential_difference(vectfield, R, O, P, O) # param: field, ref frame, 1st point, 2nd point, origin

Mastering the `sympy` library can be a useful aid for practising example problems from physics texts. Often times, these example problems come without provided solutions and `sympy` allows you to check your worked solutions, albeit with a little coding effort.